In [ ]:
import sys
import time
import numpy as np
import tkinter as tk
from collections import defaultdict
import random
from tkinter import *

In [ ]:
class Maze(tk.Tk,object):
    UNIT=80
    MAZE_H=9
    MAZE_W=9
    def __init__(self):
        super(Maze,self).__init__()
        self.action_space=['U','D','L','R']
        self.n_actions=len(self.action_space)
        self.title('maze')
        self.geometry('{0}x{1}'.format(self.MAZE_H*self.UNIT,self.MAZE_W*self.UNIT))
        self._build_maze()
    def _draw_rect(self,x,y,color):
        center=self.UNIT/2
        w=center-5
        x_=self.UNIT*x+center
        y_=self.UNIT*y+center
        return self.canvas.create_rectangle(x_-w,y_-w,x_+w,y_+w,fill=color)
    def _build_maze(self):
        h=self.MAZE_H*self.UNIT
        w=self.MAZE_W*self.UNIT
        self.canvas=tk.Canvas(self,bg="white",height=h,width=w)
        for c in range(0,w,self.UNIT):
            self.canvas.create_line(c,0,c,h)
        for r in range(0,h,self.UNIT):
            self.canvas.create_line(0,r,w,r)
        self.hells=[self._draw_rect(4,3,'black'),
                   self._draw_rect(4,4,'black'),
                   self._draw_rect(4,5,'black'),
                   self._draw_rect(4,6,'black'),
                   self._draw_rect(4,2,'black'),
                   self._draw_rect(2,1,'black'),
                   self._draw_rect(2,2,'black'),
                   self._draw_rect(2,3,'black'),
                   self._draw_rect(2,4,'black'),
                   self._draw_rect(2,5,'black'),
                   self._draw_rect(2,6,'black'),
                   self._draw_rect(5,2,'black'),
                   self._draw_rect(6,2,'black'),
                   self._draw_rect(5,6,'black'),
                   self._draw_rect(6,6,'black'),
                   self._draw_rect(6,5,'black'),
                   self._draw_rect(6,4,'black'),
                   self._draw_rect(0,0,'black'),
                   self._draw_rect(0,1,'black'),
                   self._draw_rect(0,2,'black'),
                   self._draw_rect(0,3,'black'),
                   self._draw_rect(0,4,'black'),
                   self._draw_rect(0,5,'black'),
                   self._draw_rect(0,6,'black'),
                   self._draw_rect(0,7,'black'),
                   self._draw_rect(0,8,'black'),
                   self._draw_rect(1,8,'black'),
                   self._draw_rect(2,8,'black'),
                   self._draw_rect(3,8,'black'),
                   self._draw_rect(4,8,'black'),
                   self._draw_rect(5,8,'black'),
                   self._draw_rect(6,8,'black'),
                   self._draw_rect(7,8,'black'),
                   self._draw_rect(8,8,'black'),
                   self._draw_rect(8,7,'black'),
                   self._draw_rect(8,6,'black'),
                   self._draw_rect(8,5,'black'),
                   self._draw_rect(8,4,'black'),
                   self._draw_rect(8,3,'black'),
                   self._draw_rect(8,2,'black'),
                   self._draw_rect(8,1,'black'),
                   self._draw_rect(8,0,'black'),
                   self._draw_rect(7,0,'black'),
                   self._draw_rect(6,0,'black'),
                   self._draw_rect(5,0,'black'),
                   self._draw_rect(4,0,'black'),
                   self._draw_rect(3,0,'black'),
                   self._draw_rect(2,0,'black'),
                   self._draw_rect(1,0,'black')]
        self.hell_coords=[]
        for hell in self.hells:
            self.hell_coords.append(self.canvas.coords(hell))
        self.oval=self._draw_rect(5,5,'yellow')
        self.oval1=self._draw_rect(7,7,'green')
        self.oval2=self._draw_rect(3,1,'green')
        self.oval3=self._draw_rect(2,7,'green')
        self.rect=self._draw_rect(1,1,'red')
        self.canvas.pack()
    def reset(self):
        self.update()
        time.sleep(0.05)
        self.old_s=None
        self.canvas.delete(self.rect)
        self.rect=self._draw_rect(1,1,'red')
        
        return self.canvas.coords(self.rect)
    def step(self,action):
        
        count=0
        s=self.canvas.coords(self.rect)
        base_action=np.array([0,0])
        if action==0:
            if s[1]>self.UNIT:
                count+=1
                base_action[1]-=self.UNIT
        elif action==1:
            if s[1]<(self.MAZE_H-1)*self.UNIT:
                count+=1
                base_action[1]+=self.UNIT
            
        elif action==2:
            if s[0]>self.UNIT:
                count+=1
                base_action[0]-=self.UNIT
                
        elif action==3:
            if s[0]<(self.MAZE_W-1)*self.UNIT:
                count+=1
                base_action[0]+=self.UNIT
        self.canvas.move(self.rect,base_action[0],base_action[1])
        s_=self.canvas.coords(self.rect)
        #print(s,s_,self.old_s)
        done=False
        if s_==self.canvas.coords(self.oval):
         
            reward=1000
            done=True
        elif s_==self.canvas.coords(self.oval1) or s_==self.canvas.coords(self.oval2) or s_==self.canvas.coords(self.oval3):
            reward=100
        elif count==1 and s_ not in self.hell_coords:
            reward=0
        elif s_ in self.hell_coords:
            reward=-1
            done=True
   
        
        elif self.old_s==s_:
            reward=-0.0000000000000001
        else:
            reward=-0.000000000000000000001
        self.old_s=s
        return s_,reward,done
    def render(self):
        time.sleep(0.01)
        self.update()

In [ ]:
class qlearning_Agent:
    def __init__(self,actions):
        self.time=1
        self.actions=actions
        self.learningrate=0.1
        self.discount_factor=0.9
        self.epsilon=0.6/(self.time*(0.0000002)+1)
        self.q_table=defaultdict(lambda:[0.0,0.0,0.0,0.0])
    def learn(self,state,action,reward,next_state):
        #print(self.q_table)
        current_q=self.q_table[state][action]
        new_q=reward+self.discount_factor*max(self.q_table[next_state])
        self.q_table[state][action]+=self.learningrate*(new_q-current_q)
    def get_action(self,state):
        global action
        self.time=1+self.time
        print(self.time)
        if np.random.rand()<self.epsilon:
            action=np.random.choice(self.actions)
        if np.random.rand()>=self.epsilon:
            state_action=self.q_table[state]
            action=self.arg_max(state_action)
        #print(action)
        return action
    
    def arg_max(self,state_action):
        max_index_list=[]
        max_value=state_action[0]
        for index,value in enumerate(state_action):
            if value>max_value:
                max_index_list.clear()
                max_value=value
                #print(max_value)
                max_index_list.append(index)
            elif value==max_value:
                    max_index_list.append(index)
        return random.choice(max_index_list)

In [ ]:
env=Maze()
RL=qlearning_Agent(actions=list(range(env.n_actions)))

In [ ]:
for episode in range(1000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000):
    s=env.reset()
    while True:
        env.render()
        #print(s)
        action=RL.get_action(str(s))
        s_,r,done=env.step(action)
        RL.learn(str(s),action,r,str(s_))
        s=s_
        if done:
            #print(episode)
            break